# Reto: Analítica de texto aplicada al estudio de sentimientos para la toma de decisiones


**Objetivo:**

- Construir un corpus utilizando las herramientas que has practicado a lo largo del módulo y realizar las siguientes actividades: calcular las frecuencias de longitudes de texto, frecuencias de palabras más comunes y extensión de vectores de stopwords.

- Realizar las operaciones de procesamiento básico de textos: tokenizar, aplicar stemming, remover stopwords.
- Crear gráficas para exploración de textos: histogramas, resumen de sentimientos, t-SNE o WordClouds.

- Implementar y aplicar el análisis de sentimientos a los textos que se han recopilado en el corpus

## Instrucciones
## 1. Elige uno de los *datasets* para trabajar: En este caso se eligió el dataset de datos financieros. Cuyo archivo es **FinancialTweets.zip**.

## 2. Generar un archivo para el reto, cuya estructura esté basada en los análisis solicitados.

## 3. Del archivo descargado, guarda todos sus registros en un DataFrame.

In [ ]:
# Montando google drive
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import os
print(os.listdir('/content/drive/MyDrive/Colab'))

In [ ]:
# Guardando registros en un dataframe
import pandas as pd
corpus = pd.read_csv('/content/drive/MyDrive/Colab/stockerbot-export.csv',
                     on_bad_lines='skip')
corpus.head(10)

In [ ]:
# Eliminando columnas que no se utilizarán en el análisis
corpus = corpus.drop(columns={'id','timestamp','url'})
corpus.head(10)

## 4. Corre la preparación y creación de un corpus: cálculo de frecuencias de longitudes de texto, frecuencias de palabras más comunes, extensión de vectores de stopwords.

## 5. Realiza las operaciones de procesamiento básico: tokenizar, aplicar stemming, remover stopwords

In [ ]:
# Importando bibliotecas
import regex as re
import nltk
from nltk.corpus import stopwords
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
stopwords_en = set(stopwords.words('english'))
stopwords_extra = {'inc','ltd','llc','corp'}
stopwords_all = stopwords_en.union(stopwords_extra)

# Limpiar texto de URL, de RT y de entidades HTML
def limpiar_texto(texto):
    texto = texto.lower()
    texto = re.sub(r'https\S+|www\S+', "",texto)
    texto = re.sub(r'\brt\b', '',texto)
    texto = re.sub(r'&\w+;', '',texto)
    texto = re.sub(r'[^a-z\s]', '',texto)
    return texto

# Función para tokenizar texto
def tokenize(texto):
    return texto.split()

# Función para quitar stopwords y palabras cortas
def quita_stopwords(tokens, min_len = 3 ):
    return [t for t in tokens if t not in stopwords_all and len(t) >=min_len]

# Función para preprocesar texto
def preprocesa_texto(texto):
    tokens = limpiar_texto(texto)
    tokens = tokenize(tokens)
    tokens = quita_stopwords(tokens)
    return tokens

In [ ]:
# Longitud
corpus['review_len'] = corpus['text'].astype(str).apply(len)
corpus['review_wc'] = corpus['text'].apply(lambda x: len(str(x).split()))

# Tokenizar
corpus['tokens'] = corpus['text'].astype(str).apply(
    preprocesa_texto
)
corpus['tokens_wc'] = corpus['tokens'].apply(len)
corpus.head(10)

## 6. Crea gráficas para exploración de textos: histogramas, resumen de sentimientos, t-SNE o WordClouds.

In [ ]:
# Preparación para los gráficas
tokens = [tok for tokens in corpus['tokens'] for tok in tokens]

# Función palabras más comunes
'''
La función filtra las palabras más comunes de un corpus.
'''
def palabras_comunes(lista_tokens, top=25):
    counter = Counter(lista_tokens)
    mas_comunes = counter.most_common(top)
    palabras, frecs = zip(*mas_comunes)
    sns.barplot(x=frecs,y=palabras,palette='muted')
    plt.title('Palabras más comunes')
    plt.savefig('/content/drive/MyDrive/Colab/imagenes/mas_comunes.png')
    plt.show()

# Función nube de palabras
'''
Esta función genera una nube de palabras a partir de un corpus
'''
def  nube_palabras(corpus,stopwords,color):
    stopwords = set(stopwords_all)
    wordcloud = WordCloud(
    background_color = color,
    stopwords = stopwords,
    max_words = 100,
    max_font_size = 45,
    min_font_size = 3,
    random_state = 42)

    wordcloud = wordcloud.generate(str(corpus))
    fig = plt.figure(1, figsize=(12, 12))
    plt.imshow(wordcloud)
    plt.savefig('/content/drive/MyDrive/Colab/imagenes/nube_palabras.png')
    plt.show()

In [ ]:
# Palabras más comunes
palabras_comunes(tokens)

Gráfica de las 25 palabras más comúnes:

![Palabras frecuentes](images/mas_comunes.png)

In [ ]:
# Nube de palabras
nube_palabras(corpus,stopwords_all,'white')

Gráfica de la nube de palabras:

![Nubes de palabras](images/nube_palabras.png)

In [ ]:
# Longitud de palabras
'''Se omitirán los textos con más de 100 palabras'''
sns.histplot(corpus[corpus['review_wc']<100]['review_wc'],
             bins=30,
             kde=False,
             color='blue')
plt.title('Distribución de la longitud de palabras')
plt.xlabel('Número de palabras por texto')
plt.ylabel('Frecuencia')
plt.savefig('/content/drive/MyDrive/Colab/imagenes/longitud_palabras.png')
plt.show()

Longitud de las palabras en los tweets:

![Longitud de las palabras](images/longitud_palabras.png)

In [ ]:
# Longitud de tokens
sns.histplot(corpus[corpus['tokens_wc']<30]['tokens_wc'],
             bins=30,
             kde=False,
             color='green')
plt.title('Distribución de la longitud de palabras')
plt.xlabel('Número de tokens por texto')
plt.ylabel('Frecuencia')
plt.savefig('/content/drive/MyDrive/Colab/imagenes/longitud_tokens.png')
plt.show()

Longitud de los tokens:
![Tokens](images/longitud_tokens.png)

In [ ]:
!pip install pysentimiento

In [ ]:
# Análisis de sentimiento
from pysentimiento import create_analyzer
analyzer = create_analyzer(task="sentiment", lang="en")

In [ ]:
# Desactivar wandb
import os
os.environ['WANDB_DISABLED'] = 'true'

In [ ]:
# Análisis de sentimientos
corpus['texto_limpio'] = corpus['tokens'].apply(lambda x: ' '.join(x))


In [ ]:
sentimiento = [analyzer.predict(t) for t in corpus['texto_limpio']]

In [ ]:
corpus_copia = corpus.copy()
sentimientos = []
for probabilildad in sentimiento:
    sentimientos.append(probabilildad.output)
probabilidades = {'sentimiento':sentimientos}
probabilidades = pd.DataFrame.from_dict(probabilidades)
corpus_copia = pd.concat(
    [corpus_copia,
    probabilidades.reindex(corpus_copia.index)],
    axis=1
)
corpus_copia.to_csv('corpus_sentimiento.csv',sep='\t')
corpus_copia['sentimiento'].value_counts().plot(
    kind='bar',
    color = ['#BB0000', '#0000BB', 'green'],
    title='Financial Tweets Sentiment Analysis'
)
plt.savefig('/content/drive/MyDrive/Colab/imagenes/sentimientos.png')
plt.show()

Sentimientos en los tweets:

![Sentimientos](images/sentimientos.png)

In [ ]:
# Sentimienetos por empresa
top_empresas = corpus_copia['company_names'].value_counts().nlargest(20).index
sns.countplot(
    y='company_names',
    hue='sentimiento',
    data=corpus_copia[corpus_copia['company_names'].isin(top_empresas)],
    palette='coolwarm')
plt.title("Sentimiento por empresa")
plt.savefig('/content/drive/MyDrive/Colab/imagenes/sentimiento_empresas.png')
plt.show()

Sentimientos por empresa:

![Palabras frecuentes](images/sentimiento_empresas.png)

## 7. Agrega una interpretación de resultados obtenidos del análisis realizado.

### Palabras más frecuentes
A partir del análisis de los tweets financieros, las palabras más utilizadas son "earnings, "stocks", "price", "analysts", "eps", etc. Estas se relacionan principamente en los resultados financieros de las empresas, así como el precio de las acciones y la participación de los analistas en la interpretación de estos resultados.

Así mimso con los términos *earnings*  y *EPS (earnings per share)* indica que hay un gran interés en los reportes trimestrales.

### Nube de palabras
La nube de palabras revela los términos más frecuentes dentro del corpus como *BTC, price, True, False, American y optimistic*. Estos términos reflejan un enfoque en el ámbito financiero y criptográfico, donde se discuten temas con el valor de los activos, veracidad de la información y expectativas del mercado. Así mismo, la presencia de nombres propios, como Barry Silbert, y de empresas sugiere que gran parte del discurso se vincula con actores relevantes del sector.

###  Distribución de la longitud de las palabras

La distribución de la longitud de las palabras muestra que la mayoría de los textos son cortos, entre 10 y 25 palabras por tweet. Esto es consistente con la naturaleza de la discusión financiera en redes sociales como Twitter, donde los usuarios comparten noticias, opiniones rápidas, etc.

### Análisis de Sentimientos
En cuanto al análisis de sentimientos, se identificó que predomina el sentimiento neutro, seguido por el positivo y finalmente el negativo. Este resultado es coherente con el tipo de términos encontrados, ya que lo mayoría de tweets tiene un carácter descriptivo.
Por lo tanto, el análisis revela que los tweets tienen a mantener un tono objetivo, lo que refuerza la percepción de que estas publicaciones son utilizadas para difusión y análisis especializado.

Por otra parte, en el análisis de sentimiento por empresa, predonima igualmente el sentimiento neutro, lo que coincide con el carácter informativo y objetivo de los tweets. El sentimiento positivo ocupa el segundo lugar en la mayor parte de las empresas, a excepción de Netflix.



In [ ]:
import nbformat

notebook_path = "/content/drive/MyDrive/DS_C6_SC5_GuzmanRodriguezAndres.ipynb"

# Cargar notebook
with open(notebook_path, "r", encoding="utf-8") as f:
    nb = nbformat.read(f, as_version=4)

# Limpiar solo metadata.widgets corruptos
for cell in nb.cells:
    if "metadata" in cell and "widgets" in cell.metadata:
        del cell.metadata["widgets"]

# Guardar notebook limpio
with open(notebook_path, "w", encoding="utf-8") as f:
    nbformat.write(nb, f)
